In [1]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import sys

ROOT = os.path.join(os.path.dirname(os.getcwd()), 'backend')
if ROOT not in sys.path:
    sys.path.append(ROOT)

import pandas as pd
import numpy as np


seed_number = 42
from app.dataService import globalVariable as GV
from app.dataService.utils.processSQL import process_sql, decode_sql, generate_sql
db_schema, db_names, tables = process_sql.get_schemas_from_json(
            os.path.join(GV.SPIDER_FOLDER, "tables.json"))


/home/xingbo/anaconda3/envs/chi2022/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/home/xingbo/anaconda3/envs/chi2022/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for 

In [2]:
# table meta data
with open( os.path.join(GV.SPIDER_FOLDER, "tables.json"), "r") as f:
    table_data = json.load(f)
t_df = pd.DataFrame(table_data)
t_df

,column_names,column_names_original,column_types,db_id,foreign_keys,primary_keys,table_names,table_names_original
0,"[[-1, *], [0, perpetrator id], [0, people id],...","[[-1, *], [0, Perpetrator_ID], [0, People_ID],...","[text, number, number, text, number, text, tex...",perpetrator,"[[2, 9]]","[1, 9]","[perpetrator, people]","[perpetrator, people]"
1,"[[-1, *], [0, building], [0, room number], [0,...","[[-1, *], [0, building], [0, room_number], [0,...","[text, text, text, number, text, text, number,...",college_2,"[[9, 4], [13, 4], [19, 1], [20, 2], [15, 7], [...","[1, 4, 7, 11, 15, 22, 27, 31, 37, 39, 45]","[classroom, department, course, instructor, se...","[classroom, department, course, instructor, se..."
2,"[[-1, *], [0, id], [0, city], [0, country], [0...","[[-1, *], [0, id], [0, City], [0, Country], [0...","[text, number, text, text, text, text, text, n...",flight_company,"[[20, 7], [19, 1]]","[1, 7, 13]","[airport, operate company, flight]","[airport, operate_company, flight]"
3,"[[-1, *], [0, institution id], [0, name], [0, ...","[[-1, *], [0, instID], [0, name], [0, country]...","[text, number, text, text, number, text, text,...",icfp_1,"[[11, 7], [10, 1], [9, 4]]","[1, 4, 7, 9]","[institution, authors, papers, authorship count]","[Inst, Authors, Papers, Authorship]"
4,"[[-1, *], [0, body builder id], [0, people id]...","[[-1, *], [0, Body_Builder_ID], [0, People_ID]...","[text, number, number, number, number, number,...",body_builder,"[[2, 6]]","[1, 6]","[body builder, people]","[body_builder, people]"
...,...,...,...,...,...,...,...,...
161,"[[-1, *], [0, employee ssn], [0, project numbe...","[[-1, *], [0, Essn], [0, Pno], [0, Hours], [1,...","[text, number, number, number, text, text, tex...",company_1,[],"[1, 7, 15, 19, 22, 27]","[works on, employee, department, project, depe...","[works_on, employee, department, project, depe..."
162,"[[-1, *], [0, workshop id], [0, date], [0, ven...","[[-1, *], [0, Workshop_ID], [0, Date], [0, Ven...","[text, number, text, text, text, number, numbe...",workshop_paper,"[[10, 1], [9, 5]]","[1, 5, 9]","[workshop, submission, acceptance]","[workshop, submission, Acceptance]"
163,"[[-1, *], [0, item id], [0, title], [1, a id],...","[[-1, *], [0, i_id], [0, title], [1, a_id], [1...","[text, number, text, number, number, number, n...",epinions_1,"[[5, 1], [4, 8], [11, 8], [10, 8]]","[1, 3, 8]","[item, review, useracct, trust]","[item, review, useracct, trust]"
164,"[[-1, *], [0, party id], [0, party theme], [0,...","[[-1, *], [0, Party_ID], [0, Party_Theme], [0,...","[text, number, text, text, text, text, number,...",party_host,"[[11, 1], [12, 7]]","[1, 7, 11]","[party, host, party host]","[party, host, party_host]"


In [3]:
t_df.iloc[0]

column_names             [[-1, *], [0, perpetrator id], [0, people id],...
column_names_original    [[-1, *], [0, Perpetrator_ID], [0, People_ID],...
column_types             [text, number, number, text, number, text, tex...
db_id                                                          perpetrator
foreign_keys                                                      [[2, 9]]
primary_keys                                                        [1, 9]
table_names                                          [perpetrator, people]
table_names_original                                 [perpetrator, people]
Name: 0, dtype: object

In [4]:
# table query data (just for training)
with open( os.path.join(GV.SPIDER_FOLDER, "train_spider.json"), "r") as f:
    query_data = json.load(f)
q_df = pd.DataFrame(query_data)
q_df

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...",How many heads of the departments are older th...,"[How, many, heads, of, the, departments, are, ...","{'from': {'table_units': [['table_unit', 1]], ..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","List the name, born state and age of the heads...","[List, the, name, ,, born, state, and, age, of...","{'from': {'table_units': [['table_unit', 1]], ..."
2,department_management,"SELECT creation , name , budget_in_billions ...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","List the creation year, name and budget of eac...","[List, the, creation, year, ,, name, and, budg...","{'from': {'table_units': [['table_unit', 0]], ..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...",What are the maximum and minimum budget of the...,"[What, are, the, maximum, and, minimum, budget...","{'from': {'table_units': [['table_unit', 0]], ..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...",What is the average number of employees of the...,"[What, is, the, average, number, of, employees...","{'from': {'table_units': [['table_unit', 0]], ..."
...,...,...,...,...,...,...,...
6995,culture_company,SELECT T1.company_name FROM culture_company AS...,"[SELECT, T1.company_name, FROM, culture_compan...","[select, t1, ., company_name, from, culture_co...",What are all the company names that have a boo...,"[What, are, all, the, company, names, that, ha...","{'from': {'table_units': [['table_unit', 2], [..."
6996,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...","[SELECT, T1.title, ,, T3.book_title, FROM, mov...","[select, t1, ., title, ,, t3, ., book_title, f...",Show the movie titles and book titles for all ...,"[Show, the, movie, titles, and, book, titles, ...","{'from': {'table_units': [['table_unit', 1], [..."
6997,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...","[SELECT, T1.title, ,, T3.book_title, FROM, mov...","[select, t1, ., title, ,, t3, ., book_title, f...",What are the titles of movies and books corres...,"[What, are, the, titles, of, movies, and, book...","{'from': {'table_units': [['table_unit', 1], [..."
6998,culture_company,SELECT T2.company_name FROM movie AS T1 JOIN c...,"[SELECT, T2.company_name, FROM, movie, AS, T1,...","[select, t2, ., company_name, from, movie, as,...",Show all company names with a movie directed i...,"[Show, all, company, names, with, a, movie, di...","{'from': {'table_units': [['table_unit', 1], [..."


In [14]:
q_df[q_df["db_id"] == "customers_and_addresses"].loc[6059].values

array(['customers_and_addresses',
       'SELECT avg(order_quantity) FROM order_items',
       list(['SELECT', 'avg', '(', 'order_quantity', ')', 'FROM', 'order_items']),
       list(['select', 'avg', '(', 'order_quantity', ')', 'from', 'order_items']),
       'What is the average amount of items ordered in each order?',
       list(['What', 'is', 'the', 'average', 'amount', 'of', 'items', 'ordered', 'in', 'each', 'order', '?']),
       {'from': {'table_units': [['table_unit', 6]], 'conds': []}, 'select': [False, [[5, [0, [0, 32, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}],
      dtype=object)

In [23]:
q_df["sql"][:10].values[0]

{'from': {'table_units': [['table_unit', 1]], 'conds': []},
 'select': [False, [[3, [0, [0, 0, False], None]]]],
 'where': [[False, 3, [0, [0, 10, False], None], 56.0, None]],
 'groupBy': [],
 'having': [],
 'orderBy': [],
 'limit': None,
 'intersect': None,
 'union': None,
 'except': None}

In [24]:
grouped = q_df.groupby("db_id")
g_info = [[name, len(group)] for name, group in grouped]

In [25]:
sorted(g_info, key=lambda a:a[1], reverse=True)

[['college_2', 170],
 ['college_1', 164],
 ['hr_1', 124],
 ['store_1', 112],
 ['soccer_2', 106],
 ['bike_1', 104],
 ['dorm_1', 100],
 ['hospital_1', 100],
 ['music_1', 100],
 ['music_2', 100],
 ['allergy_1', 98],
 ['movie_1', 98],
 ['flight_1', 96],
 ['driving_school', 93],
 ['cre_Doc_Tracking_DB', 90],
 ['activity_1', 88],
 ['customers_and_addresses', 88],
 ['department_store', 88],
 ['game_1', 86],
 ['network_2', 86],
 ['products_gen_characteristics', 86],
 ['chinook_1', 84],
 ['cre_Docs_and_Epenses', 84],
 ['cre_Theme_park', 84],
 ['baseball_1', 82],
 ['cre_Drama_Workshop_Groups', 82],
 ['customers_and_invoices', 82],
 ['e_learning', 82],
 ['flight_4', 82],
 ['sakila_1', 82],
 ['wine_1', 82],
 ['apartment_rentals', 80],
 ['customers_card_transactions', 80],
 ['formula_1', 80],
 ['loan_1', 80],
 ['manufactory_1', 80],
 ['tracking_grants_for_research', 78],
 ['college_3', 74],
 ['inn_1', 74],
 ['voter_2', 72],
 ['club_1', 70],
 ['csu_1', 70],
 ['election', 68],
 ['student_1', 68],
 ['

In [26]:
db_id = "customers_and_addresses"
# "department_store"

In [8]:
q_df[q_df["db_id"]==db_id]

NameError: name 'db_id' is not defined

In [28]:
t_df[t_df["db_id"]==db_id].values
# ["column_names"].values

array([[list([[-1, '*'], [0, 'address id'], [0, 'address content'], [0, 'city'], [0, 'zip postcode'], [0, 'state province county'], [0, 'country'], [0, 'other address details'], [1, 'product id'], [1, 'product details'], [2, 'customer id'], [2, 'payment method'], [2, 'customer name'], [2, 'date became customer'], [2, 'other customer details'], [3, 'customer id'], [3, 'address id'], [3, 'date address from'], [3, 'address type'], [3, 'date address to'], [4, 'customer id'], [4, 'channel code'], [4, 'active from date'], [4, 'active to date'], [4, 'contact number'], [5, 'order id'], [5, 'customer id'], [5, 'order status'], [5, 'order date'], [5, 'order details'], [6, 'order id'], [6, 'product id'], [6, 'order quantity']]),
        list([[-1, '*'], [0, 'address_id'], [0, 'address_content'], [0, 'city'], [0, 'zip_postcode'], [0, 'state_province_county'], [0, 'country'], [0, 'other_address_details'], [1, 'product_id'], [1, 'product_details'], [2, 'customer_id'], [2, 'payment_method'], [2, 'cus

In [29]:
with open( os.path.join(GV.SPIDER_FOLDER, "query_seq.json"), "r") as f:
    q_seq_data = json.load(f)

In [30]:
q_seq_data[db_id]["nl"]

[['Find the list of cities that no customer is living in.',
  'For which countries are there more than four distinct addresses listed?',
  'Find the state and country of all cities with post code starting with 4.',
  'List the countries having more than 4 addresses listed.',
  'Which city has the most customers living in?'],
 ['Find the name of all customers whose name contains "Alex".',
  'Find the customer name and date of the orders that have the status "Delivered".',
  'Find the name of customers who are living in Colorado?',
  'Find the name of the customer who made an order most recently.',
  'Return all the distinct payment methods used by customers.'],
 ['Return the the details of all products.',
  'Find the city where the most customers live.',
  'What are the state and country of all the cities that have post codes starting with 4.\\',
  'Find the name of all customers.',
  'What are the cities no customers live in?'],
 ['Which payment method is used by most customers?',
  'W

In [46]:
q_df.loc[q_df["db_id"]==db_id]["query"]

6055                  SELECT customer_name FROM customers
6056                  SELECT customer_name FROM customers
6057                       SELECT count(*) FROM customers
6058                       SELECT count(*) FROM customers
6059          SELECT avg(order_quantity) FROM order_items
                              ...                        
6138    SELECT customer_name FROM customers EXCEPT SEL...
6139    SELECT customer_name FROM customers EXCEPT SEL...
6140    SELECT customer_name FROM customers EXCEPT SEL...
6141    SELECT t1.customer_name FROM customers AS t1 J...
6142    SELECT t1.customer_name FROM customers AS t1 J...
Name: query, Length: 88, dtype: object

In [50]:
for sql in q_df.loc[q_df["db_id"]==db_id]["query"].values:
    print(sql)

SELECT customer_name FROM customers
SELECT customer_name FROM customers
SELECT count(*) FROM customers
SELECT count(*) FROM customers
SELECT avg(order_quantity) FROM order_items
SELECT avg(order_quantity) FROM order_items
SELECT customer_name FROM customers WHERE payment_method  =  "Cash"
SELECT customer_name FROM customers WHERE payment_method  =  "Cash"
SELECT date_became_customer FROM customers WHERE customer_id BETWEEN 10 AND 20
SELECT date_became_customer FROM customers WHERE customer_id BETWEEN 10 AND 20
SELECT payment_method FROM customers GROUP BY payment_method ORDER BY count(*) DESC LIMIT 1
SELECT payment_method FROM customers GROUP BY payment_method ORDER BY count(*) DESC LIMIT 1
SELECT customer_name FROM customers WHERE payment_method  =  (SELECT payment_method FROM customers GROUP BY payment_method ORDER BY count(*) DESC LIMIT 1)
SELECT customer_name FROM customers WHERE payment_method  =  (SELECT payment_method FROM customers GROUP BY payment_method ORDER BY count(*) DESC